In [ ]:
# !wget -nc https://aplj13.ifap.min-agricultura.pt/idigital.consultarFundos/pub/FEAGA_FEADER_2014.zip]
# !wget -nc https://aplj13.ifap.min-agricultura.pt/idigital.consultarFundos/pub/FEAGA_FEADER_2015.zip

In [ ]:
!wget -nc https://aplj13.ifap.min-agricultura.pt/idigital.consultarFundos/pub/FEAGA_FEADER_2016.zip

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
YEAR = 2016

In [ ]:
df = pd.read_csv('FEAGA_FEADER_%s.zip' % YEAR, delimiter=';', encoding='latin1', skiprows=1, compression='zip')
df.head()

In [ ]:
df = df.rename(columns={
        'exercicio': 'year',
        'medida': 'scheme',
        'montante': 'amount',
        'denominacao': 'recipient_name',
        'codigo_postal': 'recipient_postcode',
    })
df['distrito'] = pd.to_numeric(df['distrito'], errors='coerce')
df['concelho'] = pd.to_numeric(df['concelho'], errors='coerce')

distrito_dict = df.groupby(['distrito']).first()['descricao_distrito'].to_dict()
concelho_dict = df.groupby(['concelho']).first()['descricao_concelho'].to_dict()
df['distrito'] = df['distrito'].map(distrito_dict)
df['concelho'] = df['concelho'].map(concelho_dict)
df = df.drop(['descricao_distrito', 'descricao_concelho'], 1)
df.head()

In [ ]:
df['concelho'] = df['concelho'].fillna('')
df['distrito'] = df['distrito'].fillna('')
df.head()

In [ ]:
df['recipient_location'] = df.apply(lambda x: u'%s, %s' % (x['concelho'] or '', x['distrito']), 1)
df['amount'] = pd.to_numeric(df['amount'].str.replace(',', '.'))
df.head()

In [ ]:
len(df)

In [ ]:
from slugify import slugify
df['recipient_id'] = df.apply(lambda x: 'PT-%s-%s' % (x['recipient_postcode'], slugify(x['recipient_name'])), 1)
df = df.drop(['distrito', 'concelho'], 1)
df['currency'] = 'EUR'
df['country'] = 'PT'
df.head()

In [ ]:
df['year'].value_counts()

In [ ]:
df.to_csv('pt_%s.csv.gz' % YEAR, index=False, encoding='utf-8', compression='gzip')